In [1]:
import os, sys, django, pandas as pd , numpy as np, requests, json, re, itertools, ftfy
from django.db.models import Q
from utils import TJData, setup_env, ElisAPI
# configura notebook para se utilizar recursos do django, se necessário
setup_env() 
from api import models as tj
# objeto para interagir com a minha API
api = ElisAPI('http://0.0.0.0:8000','elisdbadmin','elisdbpassword')

### Comarca
O bloco de código abaixo foi utilizado para analisar e decidir visualmente qual planilha referente a comarcas será utilizada, a planilha mapeada por `comarca` possui a coluna `COD_SERV` possui codigos distintos dos referenciados por demais planilhas
```
com_1 = TJData.read_csv(TJData.csv['comarca'],usecols=['COD_COM','NOME','NOME_REDU','COD_TJ'])
com_2 = TJData.read_csv(TJData.csv['comarca_2'], usecols=['COD_COMA','DESC_COMA','COD_TJ','DESC_REDU'])
com = pd.merge(left=com_1,right=com_2,how='inner',left_on='NOME_REDU', right_on='DESC_REDU')
```

#### Tratamento de dados

### Serventia

``` 
serv_1 = TJData.read_csv(TJData.csv['serventia_2'], usecols=['COD_SERV','DESC_SERV','COD_COMA','DESC_ABRE','DESC_REDU'] )
serv_2 = TJData.read_csv(TJData.csv['serventias_full'], usecols=['COD_SERV','DESC_SERV','COD_COMA','DESC_ABRE','DESC_REDU'])
serv = pd.merge(left=serv_1,right=serv_2,how='inner',left_on='COD_SERV', right_on='COD_SERV')
serv_outer = pd.merge(left=serv_1,right=serv_2,how='outer',left_on='COD_SERV', right_on='COD_SERV')
```
#### Tratamento de dados

### Competencias

### Tipo Personagem

### Assunto

### Classe

### Classe assunto

### Tipo movimento

### Tipo Andamento

### Cargo

### Funcionário

### Tipo Decisão Recurso

### Tipo Ato Juiz

### Ato Juiz

### Tipo Documento

### Processo

#### Obtem as serventias com função de Juizado Especial Cível das comarcas de Bangu e Campo Grande

In [2]:
serventias_jec = pd.DataFrame(
    list(map(lambda x: (x.comarca.desc_redu,x.comarca.desc_coma, x.cod_serv, x.desc_serv),
        tj.Serventia.objects.filter(
            Q(desc_serv__contains='Juizado Especial Cível'),
            Q(comarca__desc_redu='BAN') | Q(comarca__desc_redu='CGR')
    ))),
    columns=['comarca.desc_redu','comarca.desc_coma','cod_serv','desc_serv']
)
serventias_jec

,comarca.desc_redu,comarca.desc_coma,cod_serv,desc_serv
0,BAN,Regional de Bangu,204009,Cartório do 17º Juizado Especial Cível
1,BAN,Regional de Bangu,204509,17º Juizado Especial Cível
2,BAN,Regional de Bangu,204025,Cartório do 29º Juizado Especial Cível
3,BAN,Regional de Bangu,204525,29º Juizado Especial Cível
4,CGR,Regional de Campo Grande,205020,Cartório do 26º Juizado Especial Cível
5,CGR,Regional de Campo Grande,205520,26º Juizado Especial Cível
6,CGR,Regional de Campo Grande,205507,18º Juizado Especial Cível
7,CGR,Regional de Campo Grande,205007,Cartório do 18º Juizado Especial Cível


#### Obtem a lista de planilhas para processar resultantes codigos de serventias e do intervalo de anos selecinados 

####  carrega os processos que por algum motivo não tenham sido processados anteriormente

### Andamento Processo (Sentença)

### Personagem

In [3]:
files = [TJData.csv['personagem'],TJData.csv['personagem_especial'],TJData.csv['personagem_especial_original'],TJData.csv['personagem_processo']]
num_lines = [ TJData.count_lines(file) for file in files ]
num_lines

[5990665, 180213, 199, 4155468]

In [4]:
#obtem a lista de processos cadastrados
processos_indb = list(map(lambda x : x.cod_proc,tj.Processo.objects.all()))

usecols = ['COD_PROC','COD_PERS','COD_ADV','COD_TIP_PERS','TIP_POLO']
personagem_processo = pd.DataFrame(columns=usecols).rename(columns=lambda x: x.lower())
# carrega dados de personagem processo filtrando pelos processos ja cadastrados no banco de dados
for df in TJData.read_csv(TJData.csv['personagem_processo'], usecols=usecols ,chunksize=1000000):
    df = df.rename(columns= lambda x : x.lower())
    df.cod_pers = TJData.nan_to_int(df.cod_pers)
    df.cod_adv = TJData.nan_to_int(df.cod_adv)
    df.cod_tip_pers = TJData.nan_to_int(df.cod_tip_pers)
    df = df[df.cod_proc.isin(processos_indb)]
    personagem_processo = pd.concat([personagem_processo,df])

In [5]:
# obtem tipos personagem cadastrados
tp_pers = pd.DataFrame(sorted(api.response_to_json(api.get('tipospersonagem')), key = lambda k:k['cod_tip_pers']))
# converte para string os codigos do personagem para cruzar com os cod_tip_pers
tp_pers.cod_tip_pers = TJData.nan_to_int(tp_pers.cod_tip_pers)
tp_pers.cod_tip_pers_inv = TJData.nan_to_int(tp_pers.cod_tip_pers_inv)
# filtra os personagem_processo baseado nos processos cadastrados no banco
tp_pers = tp_pers[tp_pers.cod_tip_pers.isin(list(personagem_processo.cod_tip_pers.unique()))]
# filtra os personagem_processo caso sejam advogado, escritório advocacia, personagem ativo e passivo
tp_pers = tp_pers[(tp_pers.cod_tip_pers=='137') | (tp_pers.cod_tip_pers=='0') | (tp_pers.tipo_part=='A') | (tp_pers.tipo_part=='P')].reset_index(drop=True)
tp_pers = list(tp_pers.cod_tip_pers)
# filtra personagem_processo pelos tipos selecionados
personagem_processo = personagem_processo[personagem_processo.cod_tip_pers.isin(tp_pers)].sort_values(by='cod_proc', ascending=True).reset_index(drop=True)

In [6]:
cod_pers = list(personagem_processo[~pd.isna(personagem_processo.cod_pers)].cod_pers.unique())
cod_advs = list(personagem_processo[~pd.isna(personagem_processo.cod_adv)].cod_adv.unique())

#### carrega dados codigo e nome do personagem especial (jurídica) e personagens pessoa fĩsicaa

In [7]:
# pessoa jurídica
usecols = ['COD_PERS','NOME']
personagem = TJData.read_csv(TJData.csv['personagem_especial_original'], usecols=usecols ,nrows=500)
personagem = personagem.rename(columns=lambda x: x.lower())
personagem = personagem[['cod_pers','nome']]
personagem.cod_pers = TJData.nan_to_int(personagem.cod_pers)
personagem.nome = TJData.to_utf8_bytes(personagem.nome)

# pessoa física
for df in TJData.read_csv(TJData.csv['personagem'], usecols=usecols ,chunksize=1000000):
    df = df.rename(columns= lambda x : x.lower())
    df = df.rename(columns=lambda x: x.lower())
    df = df[['cod_pers','nome']]
    df.cod_pers = TJData.nan_to_int(df.cod_pers)
    df.nome = TJData.to_utf8_bytes(df.nome)
    df = df[df.cod_pers.isin(cod_pers)]
    personagem = pd.concat([personagem,df])

# join com personagens pessoa física e jurídica com processo
personagem_processo = personagem_processo.merge(personagem, on='cod_pers', how='left')

#### carrega dados dos advogados

In [8]:
usecols = ['COD_ADV','NUM_OAB','NOME_ADV']
advogados = TJData.read_csv(TJData.csv['advogado'],usecols=usecols).rename(columns=lambda x: x.lower())
advogados.num_oab = TJData.to_utf8_bytes(advogados.num_oab)
advogados.nome_adv = TJData.to_utf8_bytes(advogados.nome_adv)
advogados.cod_adv = TJData.nan_to_int(advogados.cod_adv)
# join de advogados com personagem processo
personagem_processo = personagem_processo.merge(advogados, on='cod_adv', how='left')

#### divide as associações aos processos entre personagens (fisico ou juridico) e advogados

In [9]:
adv_usecols = ['cod_proc','cod_adv','cod_tip_pers','tip_polo','num_oab','nome_adv']
pers_usecols = ['cod_proc','cod_pers','cod_tip_pers','tip_polo','nome']
# obtem associações entre advogados e processos
advogado_processo = personagem_processo[~pd.isna(personagem_processo.cod_adv)]
advogado_processo = advogado_processo[adv_usecols]

# obtem associações entre personagens e processos
personagem_processo = personagem_processo[pd.isna(personagem_processo.cod_adv)]
personagem_processo = personagem_processo[pers_usecols]

# obtem personagem a partir das associações
personagem = personagem_processo[['cod_pers','nome']]
personagem = personagem.drop_duplicates()

# obtem detalhamento das associações entre personagem e processo
personagem_processo = personagem_processo[['cod_proc','cod_pers','cod_tip_pers','tip_polo']]

# obtem advogados a partir das associações
advogados = advogado_processo[['cod_adv','num_oab','nome_adv']]
advogados = advogados.drop_duplicates()

# obtem detalhamento das associações entre advogados e processo
advogado_processo = advogado_processo[['cod_proc','cod_adv','cod_tip_pers','tip_polo']]

((389422, 4), (275890, 2))